In [1]:
# ac/utils.py
import numpy as np
import pandas as pd
from datetime import datetime
import importlib.util
from pathlib import Path
from arcticdb import Arctic, QueryBuilder

def initialize_db(db_path):
    global ac
    ac_local = Arctic(f'lmdb://{db_path}?map_size=5MB')
    
    if not "general" in ac_local.list_libraries():
        print("Creating library 'general' where *settings and *strategies will be stored")
        library = ac_local.get_library('general', create_if_missing=True)
        index_values = ['port', 
                        's3_db_management', # False for local
                        'aws_access_id', 'aws_access_key',
                        'bucket_name','region',
                        'start_tws','username','password']
        
        data = {'Value': ["7497", # default port
                        "False", # defaul local
                        "", "", # aws_access_id, key
                        "", "", # bucket_name, region
                        "False","","" # Start TWS Automatically default False
                        ]}
        df = pd.DataFrame(data, index=index_values)
        library.write(symbol="settings",data=df)
        ac = ac_local
        return ac
    
    else: # read local settings if settings table exists
        library = ac_local.get_library('general', create_if_missing=True)
        settings_df = library.read("settings").data
        
        # if S3 is set, change ac from local to s3
        if settings_df.loc["s3_db_management","Value"] == str(True):
            region = settings_df.loc["region","Value"]
            bucket_name = settings_df.loc["bucket_name","Value"]
            id = settings_df.loc["aws_access_id","Value"]
            key = settings_df.loc["aws_access_key","Value"]
        
            ac =Arctic(f's3://s3.{region}.amazonaws.com:{bucket_name}?region={region}&access={id}&secret={key}')

            # check if "settings" exists in s3 ac
            if not "general" in ac.list_libraries():
                lib = ac.get_library('general', create_if_missing=True)
                # copy settings from local ac
                lib.write("settings", settings_df)
            elif not "settings" in ac.get_library('general', create_if_missing=True).list_symbols():
                ac.get_library('general', create_if_missing=True).write("settings", settings_df)
        else:
            ac = ac_local
        return ac

In [2]:
# ac = Arctic(f'lmdb://db?map_size=5MB')
ac = initialize_db("db")

In [3]:
ac.list_libraries()

['general', 'settings', 'spx500']

In [4]:
lib = ac.get_library('spx500')

In [5]:
lib.list_symbols()

['ALL_STOCKS']

In [7]:
df = lib.read('ALL_STOCKS').data
df

,Symbol,Open,High,Low,Close,Volume,Name,Sector,Industry,50D_SMA,200D_SMA,ATR,1M,3M,6M,12M,RS IBD,RS Rank,RS Rank 20D MA
Date,,,,,,,,,,,,,,,,,,,
2022-01-06,A,146.810340,147.905131,143.585144,147.076630,2298300.0,Agilent Technologies,Health Care,Life Sciences Tools & Services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,EMN,114.486667,115.054309,112.774442,113.667778,1034600.0,Eastman Chemical Company,Materials,Specialty Chemicals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,EMR,91.525911,92.461111,90.915159,91.831284,2477000.0,Emerson Electric,Industrials,Electrical Components & Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,ENPH,156.009995,156.229996,145.820007,151.490005,3546200.0,Enphase,Information Technology,Semiconductor Materials & Equipment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-06,EOG,86.148351,86.281229,83.774288,84.615837,5254700.0,EOG Resources,Energy,Oil & Gas Exploration & Production,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-05,JCI,56.610001,57.520000,56.230000,57.180000,4597100.0,Johnson Controls,Industrials,Building Products,52.948319,57.798674,1.209679,0.057952,0.121640,-0.133600,-0.108727,0.000952,0.162325,0.110089
2024-01-05,JKHY,166.059998,167.979996,166.059998,166.720001,567500.0,Jack Henry & Associates,Financials,Transaction & Payment Processing Services,156.048761,155.684989,2.835817,0.033410,0.125842,0.029663,-0.046345,0.235002,0.320641,0.346189
2024-01-05,JNJ,160.500000,161.339996,159.470001,161.130005,5602500.0,Johnson & Johnson,Health Care,Pharmaceuticals,152.806037,157.686314,2.058496,0.016272,0.033603,0.027313,-0.078335,0.016184,0.170341,0.147522


In [16]:
def is_last_business_day_in_df(df):
    """
    Check if the last business day is in the DataFrame's index.

    Args:
    df (pd.DataFrame): The DataFrame to check. Its index should be datetime-like.

    Returns:
    bool: True if the last business day is in the index, False otherwise.
    """
    # Ensure the DataFrame's index is a DatetimeIndex
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be a DatetimeIndex")

    # Calculate the last business day
    last_business_day = datetime.today() - pd.tseries.offsets.BDay(1)
    print(last_business_day)

    # Check if the last business day is in the DataFrame's index
    return last_business_day.normalize() in df.index

In [17]:
is_last_business_day_in_df(df)

2024-01-05 22:28:43.450155


True

In [33]:
df["active"] = True

In [34]:
lib.write('strategies',df)

VersionedItem(symbol='strategies', library='general', data=n/a, version=4, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [35]:
df = lib.read('strategies').data

In [36]:
df

,name,filename,description,target_weight,min_weight,max_weight,params,active
SVIX,Short VIX Future,SVIX.py,If the Volatility Risk Premium is positive and...,0.05,0.02,0.08,"{'target_weight': 0.05, 'min_weight': 0.02, 'm...",True
test1,test,test.py,,1,1,1,[Errno 2] No such file or directory: '/Users/j...,True
